In [14]:
import camelot
import numpy as np
import pandas as pd
import json
import os
#import pdb

df_str_cc_lmt,df_str_due_dt,df_str_cc_stmt,concat_df_lmt,concat_df_datconcat_df_smt = '','','','',''
df_cc_lmt, df_due_dt, df_cc_stmt = [],[],[]

pd.options.mode.chained_assignment = None

In [15]:
with open('config.json') as f:
    config = json.load(f)

in_file = config['cc_pdf_input_location']
password = config['cc_password']
file_list = [x.split('.')[0] for x in os.listdir(in_file['cc_pdf_input_location'])]
out_file = config['cc_csv_output_location']

for i in range(len(file_list)):
    df_cc_lmt.append(str("df_cc_lmt{0}".format(i)))
    df_str_cc_lmt += """df_cc_lmt[{0}],""".format(i)

    df_due_dt.append(str("df_due_dt{0}".format(i)))
    df_str_due_dt += """df_due_dt[{0}],""".format(i)

    df_cc_stmt.append(str("df_cc_stmt{0}".format(i)))
    df_str_cc_stmt += """df_cc_stmt[{0}],""".format(i)

concat_df_lmt = """concatenated_df = pd.concat(["""+df_str_cc_lmt[:-1]+"""],ignore_index=True,axis = 0)"""
concat_df_dat = """concatenated_df = pd.concat(["""+df_str_due_dt[:-1]+"""],ignore_index=True,axis = 0)"""
concat_df_smt = """concatenated_df = pd.concat(["""+df_str_cc_stmt[:-1]+"""],ignore_index=True,axis = 0)"""

In [16]:
def cc_stmt_decode(pdf_path,out_path,password,file_name):
    global pdf_due_df,pdf_cc_limit_df,pdf_cc_stmt_df
    cc_stmt_lst = []
    read_multi_table = 'NO'
    print('read_table at the top',read_multi_table)

    tables = camelot.read_pdf(pdf_path,pages='all',password=password)
    temp_table2 = tables[2].df
    print(temp_table2[0][0])
    if temp_table2[0][0] == 'Date':
        read_multi_table = 'Yes'
    print('read_table after val check',read_multi_table)

    pdf_due_df = pd.DataFrame(tables[0].df.iloc[1:3])
    pdf_due_df = pdf_due_df.rename(columns=pdf_due_df.iloc[0]).drop(pdf_due_df.index[0])

    pdf_cc_limit_df = pd.DataFrame(tables[0].df.iloc[3:])
    pdf_cc_limit_df = pdf_cc_limit_df.rename(columns=pdf_cc_limit_df.iloc[0]).drop(pdf_cc_limit_df.index[0])

    if read_multi_table == 'Yes':
        pdf_cc_stmt_df = pd.concat([tables[1].df.iloc[0:],tables[2].df.iloc[1:]],ignore_index=True,axis = 0)
    else:
        pdf_cc_stmt_df = pd.DataFrame(tables[1].df.iloc[0:])

    pdf_cc_stmt_df = pdf_cc_stmt_df[pdf_cc_stmt_df[0] != 'Domestic Transactions']
    pdf_cc_stmt_df = pdf_cc_stmt_df.rename(columns=pdf_cc_stmt_df.iloc[0]).drop(pdf_cc_stmt_df.index[0])

    cc_stmt_lst_dict = pdf_cc_stmt_df.to_dict()
    if 'Feature Reward\nPoints' in cc_stmt_lst_dict.keys():
        del cc_stmt_lst_dict['Feature Reward\nPoints']
    dic_key = list(cc_stmt_lst_dict.keys())

    for i in cc_stmt_lst_dict:
        for j in cc_stmt_lst_dict[i]:
            cc_stmt_lst.append(str(cc_stmt_lst_dict[i][j]).split('\n'))

    cc_stmt_lst = [[x for x in sub if x != ' ' and x != 'DHEERAJ PATEL' and x!= 'RUPAL  PATIDAR'] for sub in cc_stmt_lst]
    
    if read_multi_table == 'Yes':
        pdf_cc_stmt_df1 = pd.DataFrame({dic_key[0]:cc_stmt_lst[0],dic_key[1]:cc_stmt_lst[2],dic_key[2]:cc_stmt_lst[4],'File_name':file_name})
        pdf_cc_stmt_df2 = pd.DataFrame({dic_key[0]:cc_stmt_lst[1],dic_key[1]:cc_stmt_lst[3],dic_key[2]:cc_stmt_lst[5],'File_name':file_name})
        pdf_cc_stmt_df = pd.concat([pdf_cc_stmt_df1,pdf_cc_stmt_df2],ignore_index=True,axis = 0)
    else:
        pdf_cc_stmt_df = pd.DataFrame({dic_key[0]:cc_stmt_lst[0],dic_key[1]:cc_stmt_lst[1],dic_key[2]:cc_stmt_lst[2],'File_name':file_name})

    with pd.ExcelWriter(out_path) as writer:
        pdf_due_df.to_excel(writer, sheet_name='CC due date')  
        pdf_cc_limit_df.to_excel(writer, sheet_name='CC limit')
        pdf_cc_stmt_df.to_excel(writer, sheet_name='CC stmt')

    return pdf_due_df,pdf_cc_limit_df,pdf_cc_stmt_df

In [17]:
#cc_stmt_decode_test(r'C:\Users\pc\Desktop\Dheeraj\Finance analysis\pdf\credit card\input\4854XXXXXXXXXX57_16-01-2024.pdf',out_file['cc_csv_output_location']+'4854XXXXXXXXXX57_16-01-2024.xlsx',password['cc_password'])

In [18]:
for i in range(len(file_list)):
    print('File number: ',i+1)
    print('**In-put file {0} read and conversion to xlsx started**'.format(in_file['cc_pdf_input_location']+file_list[i]+'.pdf'))
    df_due_dt[i],df_cc_lmt[i],df_cc_stmt[i] = cc_stmt_decode(in_file['cc_pdf_input_location']+file_list[i]+'.pdf',out_file['cc_csv_output_location']+file_list[i]+'.xlsx',password['cc_password'],file_list[i])
    #df[i] = pd.DataFrame(pdf_cc_stmt_df)
    print('**Output file {0} created**'.format(out_file['cc_csv_output_location']+file_list[i]+'.xlsx'))
    print(' ')

File number:  1
**In-put file C:\Users\pc\Desktop\Dheeraj\Finance analysis\pdf\credit card\input\4854XXXXXXXXXX57_16-01-2022.pdf read and conversion to xlsx started**
read_table at the top NO
Other offers on your Credit Card
read_table after val check NO
**Output file C:\Users\pc\Desktop\Dheeraj\Finance analysis\pdf\credit card\output\4854XXXXXXXXXX57_16-01-2022.xlsx created**
 
File number:  2
**In-put file C:\Users\pc\Desktop\Dheeraj\Finance analysis\pdf\credit card\input\4854XXXXXXXXXX57_16-01-2023.pdf read and conversion to xlsx started**
read_table at the top NO
Other offers on your Credit Card
read_table after val check NO
**Output file C:\Users\pc\Desktop\Dheeraj\Finance analysis\pdf\credit card\output\4854XXXXXXXXXX57_16-01-2023.xlsx created**
 
File number:  3
**In-put file C:\Users\pc\Desktop\Dheeraj\Finance analysis\pdf\credit card\input\4854XXXXXXXXXX57_16-01-2024.pdf read and conversion to xlsx started**
read_table at the top NO
Date
read_table after val check Yes
**Output

In [25]:
print(concat_df_lmt)

concatenated_df = pd.concat([df_cc_lmt[0],df_cc_lmt[1],df_cc_lmt[2],df_cc_lmt[3],df_cc_lmt[4],df_cc_lmt[5],df_cc_lmt[6],df_cc_lmt[7],df_cc_lmt[8],df_cc_lmt[9],df_cc_lmt[10],df_cc_lmt[11],df_cc_lmt[12],df_cc_lmt[13],df_cc_lmt[14],df_cc_lmt[15],df_cc_lmt[16],df_cc_lmt[17],df_cc_lmt[18],df_cc_lmt[19],df_cc_lmt[20],df_cc_lmt[21],df_cc_lmt[22],df_cc_lmt[23],df_cc_lmt[24],df_cc_lmt[25],df_cc_lmt[26],df_cc_lmt[27],df_cc_lmt[28],df_cc_lmt[29],df_cc_lmt[30],df_cc_lmt[31],df_cc_lmt[32]],ignore_index=True,axis = 0)


In [24]:
concatenated_df

,Credit Limit,Available Credit Limit,Available Cash Limit
0,"5,11,000","30,020","1,68,477"
1,"6,13,000","5,60,396","2,45,200"
2,"6,13,000","4,89,224","2,45,200"
3,"5,11,000","4,99,612","2,04,400"
4,"5,11,000","4,97,716","2,04,400"
5,"6,13,000","6,10,008","2,45,200"
6,"6,13,000","5,59,541","2,45,200"
7,"5,11,000","5,07,720","2,04,400"
8,"5,11,000","2,88,667","1,53,300"
9,"6,13,000","4,83,644","2,45,200"


In [20]:
#df_in_yrly_stmt_dr.to_csv(out_file['csv_output_location']+'all_deposit.csv',header=True,index=False)
#%reset -f